In [71]:
import numpy as np
import math
from scipy.stats import nbinom

def truncNegBin_PDF(y, n, p):
    one_f_zero = float(1 - nbinom.pmf(0, n, p))
    return nbinom.pmf(y, n, p) / one_f_zero

def truncNegBin_CDF(y, n, p):
    f_zero = nbinom.pmf(0, n, p)
    if y > 0:
        return (nbinom.cdf(y, n, p) - nbinom.cdf(0, n, p)) / (1 - f_zero)
    else:
        return 0
    
def truncNegBin_CDF2(a,n,p):
    p=float(p)
    if a <= 0:
        return 0
    else:
        pdf_array = np.array([truncNegBin_PDF(yi, n, p) for yi in range(1, a+1)])
        return np.sum(pdf_array)

def truncNegBin_PPF(x, n, p, epsilon=1e-6, max_iterations=100):
    # Define the range of y where the solution might exist
    lower_bound = 0
    upper_bound = 1000000000  # Adjust this based on the expected range of y

    # Bisection method
    for _ in range(max_iterations):
        y = (lower_bound + upper_bound) / 2
        cdf_value = truncNegBin_CDF(y, n, p)

        if abs(cdf_value - x) < epsilon:
            return np.ceil(y)  # Found a good approximation

        if cdf_value < x:
            lower_bound = y
        else:
            upper_bound = y

    # Return the best approximation if max_iterations is reached
    return np.ceil(y)

mean = 1000
var = 8000

n = (mean**2) / (var - mean) # equivalent to r
p = mean / var


print(nbinom.cdf(10, n, p))
print(truncNegBin_CDF(10, n, p))
print(truncNegBin_CDF2(10, n, p))

3.664901396367409e-115
3.664901396367399e-115
3.664901396367431e-115


#### Proof that the inverse function leads to the same results as the naive way of calculating the quantiles

In [74]:
from scipy.stats import nbinom



# Example usage
mean = 100
var = 80000

n = (mean**2) / (var - mean) # equivalent to r
p = mean / var

x = 0.999
inverse_value = truncNegBin_PPF(x, n, p)
print(inverse_value)

y_values = range(1, int(inverse_value)+100)
probabilities = np.array([truncNegBin_PDF(yi, n, p) for yi in y_values])
cdf_array = np.cumsum(probabilities)
quantile = np.argmax(cdf_array >= list([x])) + 1
print(quantile)

3248.453140258789
3249


In [62]:
# calculation of quantiles
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies
trunc_nbinom_quantiles = np.array([truncNegBin_PPF(quantile, n, p) for quantile in quantiles])

In [63]:
trunc_nbinom_quantiles

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
      